In [1]:
import pandas as pd
from collections import Counter, defaultdict

In [3]:
pred_asp = pd.read_csv('our_dev_pred_aspects.txt', index_col= 0)
pred_asp = pred_asp.rename(columns={'mention':'aspect'})

In [4]:
pred_asp['sentiment'].value_counts()

positive    694
neutral     315
negative    181
Name: sentiment, dtype: int64

In [5]:
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/dev_cats.txt
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/dev_pred_cats.txt

--2021-12-27 13:05:55--  https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/dev_cats.txt
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_cats.txt [following]
--2021-12-27 13:05:55--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_cats.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7400 (7.2K) [text/plain]
Saving to: ‘dev_cats.txt’

dev_cats.txt        100%[===================>]   7.23K  --.-KB/s    in 0.001s  

2021-12-27 13:05:55 (8.58 MB/s) - ‘dev_cats.txt’ saved [7400/7400]

--2021-1

In [6]:
def get_cat_sentiment(asp_cats):
      CATEGORIES = ['Whole', 'Interior', 'Service', 'Food', 'Price']
      
      texts = set(asp_cats['text_id'])
      for text in texts:
        temp = asp_cats[asp_cats['text_id'] == text].to_dict('records')
        asp_counter = defaultdict(Counter)
        for row in temp:
            asp_counter[row['category']][row['sentiment']] += 1
       
        for c in CATEGORIES:
            if not asp_counter[c]:
                s = 'absence'
            elif len(asp_counter[c]) == 1:
                s = asp_counter[c].most_common(1)[0][0]
                
            else:
                pos_score = 0
                neg_score = 0
                neu_score = 0
                total = 0
                for sent, count in asp_counter[c].items():
                  total += count
                  if sent == 'positive':
                    pos_score += count
                  elif sent == 'negative':
                    neg_score += count
                  elif sent == 'neutral':
                    neu_score += count
                
                if pos_score/total > 0.6:
                  s = 'positive'
                elif neg_score/total > 0.6:
                  s = 'negative'
                elif neu_score/total > 0.6:
                  s = 'neutral'
                
                else:
                  s = 'both'

            yield text, c, s

In [7]:
with open('our_dev_pred_cats.txt', 'w') as f:
        for idx, c, s in get_cat_sentiment(pred_asp):
            print(idx, c, s, sep="\t", file=f)

In [10]:
test = pd.read_csv('our_dev_pred_cats.txt', delimiter='\t', names=['text_id', 'cat', 'sent'])

In [11]:
test

,text_id,cat,sent
0,9216,Whole,negative
1,9216,Interior,negative
2,9216,Service,negative
3,9216,Food,negative
4,9216,Price,absence
...,...,...,...
350,13823,Whole,neutral
351,13823,Interior,absence
352,13823,Service,neutral
353,13823,Food,neutral


In [13]:
import numpy as np

In [14]:
gold = pd.read_csv('dev_cats.txt', delimiter='\t', names=['text_id', 'cat', 'sent'])
gold['comparison'] = np.where(gold['sent'] == test['sent'], 'True', 'False')

In [21]:
false = gold[gold['comparison'] == 'False']
true = gold[gold['comparison'] == 'True']

In [28]:
false['sent'].value_counts()

positive    108
absence      58
both         37
negative     33
neutral       3
Name: sent, dtype: int64

In [29]:
true['sent'].value_counts()

positive    106
neutral       5
negative      3
absence       1
both          1
Name: sent, dtype: int64

In [8]:
gold_test_cats_path = "dev_cats.txt"
pred_test_cats_path = "our_dev_pred_cats.txt"
with open(gold_test_cats_path) as gc, open(pred_test_cats_path) as pc:
    gold_labels = set(gc.readlines())
    pred_labels = set(pc.readlines())
    print(
        "Overall sentiment accuracy:",
        len(gold_labels & pred_labels) / len(gold_labels)
    )

Overall sentiment accuracy: 0.6253521126760564


In [9]:
gold_test_cats_path = "dev_cats.txt"
pred_test_cats_path = "dev_pred_cats.txt"
with open(gold_test_cats_path) as gc, open(pred_test_cats_path) as pc:
    gold_labels = set(gc.readlines())
    pred_labels = set(pc.readlines())
    print(
        "Overall sentiment accuracy:",
        len(gold_labels & pred_labels) / len(gold_labels)
    )

Overall sentiment accuracy: 0.523943661971831
